## Pre-requisites

Ensure following tools are installed before proceeding

1. kubectl - The Kubernetes command-line tool
2. helm - The package manager for Kubernetes
3. curl - For testing API endpoints (installed by default on most systems)
4. AWS CLI - AWS command line interface

### Setup Kubernetes Cluster on Amazon EKS
1. Follow the setup intstructions in this [Amazon EKS workshop](https://www.eksworkshop.com/docs/introduction/setup/your-account/) to create a Kubernetes cluster.
2. Note the newly created **Amazon EKS Cluster name** .
    

### Update Kubeconfig file

Replace both AWS region and EKS cluster name below.

In [ ]:
!aws eks --region <REPLACE_AWS_REGION> update-kubeconfig --name <REPLACE_EKS_CLUSTER_NAME>

In [ ]:
# Import required libraries

import os
import subprocess
import requests
import json


### Install envoy gagteway system

In [ ]:
!helm upgrade -i eg oci://docker.io/envoyproxy/gateway-helm \
    --version v0.0.0-latest \
    --namespace envoy-gateway-system \
    --create-namespace

!kubectl wait --timeout=2m -n envoy-gateway-system deployment/envoy-gateway --for=condition=Available

### Install envoy AI Gateway

In [ ]:
!helm upgrade -i aieg oci://docker.io/envoyproxy/ai-gateway-helm \
    --version v0.0.0-latest \
    --namespace envoy-ai-gateway-system \
    --create-namespace

!kubectl wait --timeout=2m -n envoy-ai-gateway-system deployment/ai-gateway-controller --for=condition=Available

### Configure Envoy Gateway

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/envoyproxy/ai-gateway/main/manifests/envoy-gateway-config/redis.yaml
!kubectl apply -f https://raw.githubusercontent.com/envoyproxy/ai-gateway/main/manifests/envoy-gateway-config/config.yaml
!kubectl apply -f https://raw.githubusercontent.com/envoyproxy/ai-gateway/main/manifests/envoy-gateway-config/rbac.yaml

!kubectl rollout restart -n envoy-gateway-system deployment/envoy-gateway

!kubectl wait --timeout=2m -n envoy-gateway-system deployment/envoy-gateway --for=condition=Available

### Verify Pods

In [ ]:
!kubectl get pods -n envoy-ai-gateway-system

In [ ]:
!kubectl get pods -n envoy-gateway-system

## Basic Configuration

In [ ]:
# Donwload the configuration template

!curl -O https://raw.githubusercontent.com/envoyproxy/ai-gateway/main/examples/basic/basic.yaml

In [ ]:
!kubectl apply -f https://raw.githubusercontent.com/envoyproxy/ai-gateway/main/examples/basic/basic.yaml

## Wait for Pods to be ready

In [ ]:
!kubectl wait pods --timeout=2m \
    -l gateway.envoyproxy.io/owning-gateway-name=envoy-ai-gateway-basic \
    -n envoy-gateway-system \
    --for=condition=Ready

## Configure GATEWAY_URL

Verify the assigned external IP

In [ ]:
!kubectl get svc -n envoy-gateway-system \
    --selector=gateway.envoyproxy.io/owning-gateway-namespace=default,gateway.envoyproxy.io/owning-gateway-name=envoy-ai-gateway-basic

## Use external IP

In [ ]:


# Run kubectl command and capture the output
gateway_url = subprocess.check_output(
    "kubectl get gateway/envoy-ai-gateway-basic -o jsonpath='{.status.addresses[0].value}'",
    shell=True
).decode().strip()

# Set the environment variable
os.environ['GATEWAY_URL'] = gateway_url

# Print the value to verify
print(f"GATEWAY_URL set to: {gateway_url}")


## Testing with model backend

### Ensure the following
1. Update `AWS_ACCESS_KEY_ID` and  `AWS_SECRET_ACCESS_KEY` in basic.yaml file
2. Enable access to "amazon.nova-lite-v1:0" in the `us-east-1` region
3. Update `OPENAI_API_KEY` in basic.yaml file
4. Update `x-ai-eg-model` value to `us.amazon.nova-lite-v1:0` from `us.meta.llama3-2-1b-instruct-v1:0` in basic.yaml file.

Note - For production use, consider use of AWS secrets manager to store credentials.

Lets apply the configuration

In [ ]:
!kubectl apply -f basic.yaml

!kubectl wait pods --timeout=2m \
  -l gateway.envoyproxy.io/owning-gateway-name=envoy-ai-gateway-basic \
  -n envoy-gateway-system \
  --for=condition=Ready

### Next, lets test the configuration

1. First we invoke `Amazon Nova Lite model` on Amazon Bedrock to generate a summary of a customer review
2. We pass the output of the first LLM call to OpenAI `gpt-4o-mini` model to generate overall sentiment of the customer review

In [ ]:
review = """I recently purchased the Smart Coffee Maker from this website, and I must say, it has been a total game-changer for my morning routine! The sleek design fits perfectly on my kitchen counter, and the touchscreen interface is incredibly user-friendly. The coffee it brews is always hot and flavorful, and I love that I can schedule it to start brewing before I wake up.

What really impressed me, though, was the customer service. I had a minor issue with the app connectivity, and the support team responded promptly and resolved the problem within hours. The quality of the product and the service has exceeded my expectations. I find it easy to use, and the great tasting coffee is a bonus. Overall, I highly recommend this coffee maker to anyone looking to upgrade their morning coffee experience.

I don't have any complaints so far, which is a testament to how well this product has performed."""

prompt = f"Analyze the following product review:\n\n{review}\n\nProvide a short summary review"

# Define the payload
payload = {
    "model": "us.amazon.nova-lite-v1:0",
    "messages": [
        {
            "role": "user",
            "content":  prompt
        }
    ]
}

payload = json.dumps(payload)

print("Bedrock Payload:: " + payload)

# Get the GATEWAY_URL from environment variable
gateway_url = "http://"+os.environ.get('GATEWAY_URL')

url = f"{gateway_url}/v1/chat/completions"

# Set the headers
headers = {
    'Content-Type': 'application/json'
}


# Make the POST request to Bedrock
response = requests.post(url, headers=headers, data=payload)

# Store the response
response_data = response.json()

# Print the response (optional)
print(json.dumps(response_data, indent=2))

content = response_data['choices'][0]['message']['content']

print("\n\nGenerated review summary::\n" + content)


Now we use the summary generated using Amazon Bedrock model and pass it on to OpenAI.

In [ ]:
# Build the prompt based on content generated in the first LLM call to Bedrock
prompt = f"Based on this product review summary:\n\n{content}\n\nWhat is the overall sentiment of the review?"


# Define the payload
payload = {
    "model": "gpt-4o-mini",
    "messages": [
        {
            "role": "user",
            "content":  prompt
        }
    ]
}

payload = json.dumps(payload)

print("OpenAI Payload:: " + payload)

# Get the GATEWAY_URL from environment variable
gateway_url = "http://"+os.environ.get('GATEWAY_URL')

url = f"{gateway_url}/v1/chat/completions"

# Set the headers
headers = {
    'Content-Type': 'application/json'
}

# Make the POST request to OpenAI
response = requests.post(url, headers=headers, data=payload)

# Store the response
response_data = response.json()

# Print the response (optional)
print(json.dumps(response_data, indent=2))

content = response_data['choices'][0]['message']['content']

print("\n\nSentiment Analysis::\n" + content)

### Run following command to view controller logs

In [ ]:
!kubectl logs -n envoy-ai-gateway-system deployment/ai-gateway-controller